## Required Packages

Ensure you have the following Python packages installed to run this notebook:

- `numpy`: For numerical operations.
- `gurobipy`: The Python API for Gurobi optimizer.

You can install these packages via pip and system commands if you don't have them already.


In [ ]:
#!pip install gurobipy  # install gurobipy, if not already installed

In [ ]:
!pip install gurobipy

## Utilities

In [2]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import itertools
import random
import string
import time
import os

**Read input matrix (image)**

In [3]:
def read_data(file_name):
    # Read data from file and store in a matrix, and manage M, N, and mapping
    matrix = np.loadtxt(file_name)
    M, N = matrix.shape

    mapping = {}
    _edge_idx = 0

    # Generate keys for subsets of a 2x2 block of size 2, 3, and 4
    for i in range(M - 1):
        for j in range(N - 1):
            arr = [(i, j), (i, j + 1), (i + 1, j), (i + 1, j + 1)]
            # Generate combinations of size 2, 3, and 4
            for size in range(2, 5):  # Only combinations of size 2, 3, and 4
                for subset in itertools.combinations(arr, size):
                    key = ','.join(f"{pair[0]},{pair[1]}" for pair in subset) + ','
                    if key not in mapping:
                        mapping[key] = _edge_idx
                        _edge_idx += 1

    return matrix, mapping

#clean_matrix, mapping = read_data('/content/CEN_15.txt')

In [6]:
clean_matrix, mapping = read_data('/content/CEN_15.txt')

In [7]:
def compute_potential(pattern_potential):
    # Calculate objective coefficients
    objective_coefficients = [0.0] * 15
    temp = [
        pattern_potential[1] - pattern_potential[0],  # xi
        pattern_potential[0] - 2 * pattern_potential[1] + pattern_potential[2],  # xixj
        -pattern_potential[0] + 4 * pattern_potential[1] - 2 * pattern_potential[2] - pattern_potential[3],  # xixjxk
        2 * pattern_potential[0] - 8 * pattern_potential[1] + 4 * pattern_potential[2] + 2 * pattern_potential[3],  # term 4
        pattern_potential[0] - 2 * pattern_potential[1] + pattern_potential[3]  # xixj alternative
    ]

    for i in range(4):
        objective_coefficients[i] = temp[0]
    for i in range(4, 6):
        objective_coefficients[i] = temp[4]
    for i in range(6, 10):
        objective_coefficients[i] = temp[1]
    for i in range(10, 14):
        objective_coefficients[i] = temp[2]
    objective_coefficients[14] = temp[3]

    return objective_coefficients

In [8]:
pattern_potential = [10,20,30,40]
objective_coefficients = compute_potential(pattern_potential)
print(objective_coefficients)

[10, 10, 10, 10, 10, 10, 0, 0, 0, 0, -30, -30, -30, -30, 60]


Use Gurobi WLS license

In [ ]:
params = {
    "WLSACCESSID": "",
    "WLSSECRET": "",
    "LICENSEID": ,
}

env = gp.Env(params=params)

## Define IPs and LPs

In [10]:
def e(mapping, *args):
    """ Helper function to convert indices into mapping keys. """
    filtered_indices = [arg for arg in args if arg != (-1, -1)]
    filtered_indices.sort(key=lambda x: (x[0], x[1]))
    key = ','.join(f"{idx[0]},{idx[1]}" for idx in filtered_indices) + ','
    if key not in mapping:
        print("Key Error:", key)
        print("Available keys:", mapping.keys())
    return mapping.get(key, 'Default')

def define_std_IP_model(matrix, mapping):
    M, N = matrix.shape
    model = gp.Model(env=env)

    x = [[model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}") for j in range(N)] for i in range(M)]
    y = [model.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in range(len(mapping))]

    for i in range(M - 1):
      for j in range(N - 1):
          v1, v2, v3, v4 = (i, j), (i, j + 1), (i + 1, j), (i + 1, j + 1)

          y_v1_v2_v3 = y[e(mapping, v1, v2, v3)]
          y_v1_v2_v4 = y[e(mapping, v1, v2, v4)]
          y_v1_v3_v4 = y[e(mapping, v1, v3, v4)]
          y_v2_v3_v4 = y[e(mapping, v2, v3, v4)]
          y_v1_v2_v3_v4 = y[e(mapping, v1, v2, v3, v4)]
          y_v1_v2 = y[e(mapping, v1, v2)]
          y_v1_v3 = y[e(mapping, v1, v3)]
          y_v1_v4 = y[e(mapping, v1, v4)]
          y_v2_v3 = y[e(mapping, v2, v3)]
          y_v2_v4 = y[e(mapping, v2, v4)]
          y_v3_v4 = y[e(mapping, v3, v4)]

          model.addConstr(y_v1_v2 <= x[i][j])
          model.addConstr(y_v1_v2 <= x[i][j + 1])
          model.addConstr(y_v1_v2 >= x[i][j] + x[i][j + 1] - 1)
          model.addConstr(y_v1_v3 <= x[i][j])
          model.addConstr(y_v1_v3 <= x[i + 1][j])
          model.addConstr(y_v1_v3 >= x[i][j] + x[i + 1][j] - 1)
          model.addConstr(y_v1_v4 <= x[i][j])
          model.addConstr(y_v1_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v1_v4 >= x[i][j] + x[i + 1][j + 1] - 1)
          model.addConstr(y_v2_v3 <= x[i][j + 1])
          model.addConstr(y_v2_v3 <= x[i + 1][j])
          model.addConstr(y_v2_v3 >= x[i][j + 1] + x[i + 1][j] - 1)
          model.addConstr(y_v2_v4 <= x[i][j + 1])
          model.addConstr(y_v2_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v2_v4 >= x[i][j + 1] + x[i + 1][j + 1] - 1)
          model.addConstr(y_v3_v4 <= x[i + 1][j])
          model.addConstr(y_v3_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v3_v4 >= x[i + 1][j] + x[i + 1][j + 1] - 1)
          model.addConstr(y_v1_v2_v3 <= x[i][j])
          model.addConstr(y_v1_v2_v3 <= x[i][j + 1])
          model.addConstr(y_v1_v2_v3 <= x[i + 1][j])
          model.addConstr(y_v1_v2_v3 >= x[i][j] + x[i][j + 1] + x[i + 1][j] - 2)
          model.addConstr(y_v1_v2_v4 <= x[i][j])
          model.addConstr(y_v1_v2_v4 <= x[i][j + 1])
          model.addConstr(y_v1_v2_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v1_v2_v4 >= x[i][j] + x[i][j + 1] + x[i + 1][j + 1] - 2)
          model.addConstr(y_v1_v3_v4 <= x[i][j])
          model.addConstr(y_v1_v3_v4 <= x[i + 1][j])
          model.addConstr(y_v1_v3_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v1_v3_v4 >= x[i][j] + x[i + 1][j] + x[i + 1][j + 1] - 2)
          model.addConstr(y_v2_v3_v4 <= x[i][j + 1])
          model.addConstr(y_v2_v3_v4 <= x[i + 1][j])
          model.addConstr(y_v2_v3_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v2_v3_v4 >= x[i][j + 1] + x[i + 1][j] + x[i + 1][j + 1] - 2)
          model.addConstr(y_v1_v2_v3_v4 <= x[i][j])
          model.addConstr(y_v1_v2_v3_v4 <= x[i][j + 1])
          model.addConstr(y_v1_v2_v3_v4 <= x[i + 1][j])
          model.addConstr(y_v1_v2_v3_v4 <= x[i + 1][j + 1])
          model.addConstr(y_v1_v2_v3_v4 >= x[i][j] + x[i][j + 1] + x[i + 1][j] + x[i + 1][j + 1] - 3)

    return model,x,y

def define_clique_IP_model(matrix, mapping):
    M, N = matrix.shape
    model = gp.Model(env=env)

    x = [[model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{j}") for j in range(N)] for i in range(M)]
    y = [model.addVar(vtype=GRB.BINARY, name=f"y_{i}") for i in range(len(mapping))]

    # Using clique LP as constraints
    for i in range(M - 1):
      for j in range(N - 1):
          v1, v2, v3, v4 = (i, j), (i, j + 1), (i + 1, j), (i + 1, j + 1)

          y_v1_v2_v3 = y[e(mapping, v1, v2, v3)]
          y_v1_v2_v4 = y[e(mapping, v1, v2, v4)]
          y_v1_v3_v4 = y[e(mapping, v1, v3, v4)]
          y_v2_v3_v4 = y[e(mapping, v2, v3, v4)]
          y_v1_v2_v3_v4 = y[e(mapping, v1, v2, v3, v4)]
          y_v1_v2 = y[e(mapping, v1, v2)]
          y_v1_v3 = y[e(mapping, v1, v3)]
          y_v1_v4 = y[e(mapping, v1, v4)]
          y_v2_v3 = y[e(mapping, v2, v3)]
          y_v2_v4 = y[e(mapping, v2, v4)]
          y_v3_v4 = y[e(mapping, v3, v4)]
          model.addConstr(y_v1_v2_v3 - y_v1_v2_v3_v4 >= 0)
          model.addConstr(y_v1_v2_v4 - y_v1_v2_v3_v4 >= 0)
          model.addConstr(y_v1_v3_v4 - y_v1_v2_v3_v4 >= 0)
          model.addConstr(y_v2_v3_v4 - y_v1_v2_v3_v4 >= 0)
          model.addConstr(-y_v1_v2 + y_v1_v2_v3 + y_v1_v2_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-y_v1_v3 + y_v1_v2_v3 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-y_v1_v4 + y_v1_v2_v4 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-y_v2_v3 + y_v1_v2_v3 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-y_v2_v4 + y_v1_v2_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-y_v3_v4 + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
          model.addConstr(-x[i][j] + y_v1_v2 + y_v1_v3 + y_v1_v4 - y_v1_v2_v3 - y_v1_v2_v4 - y_v1_v3_v4 + y_v1_v2_v3_v4 <= 0)
          model.addConstr(-x[i][j+1] + y_v1_v2 + y_v2_v3 + y_v2_v4 - y_v1_v2_v3 - y_v1_v2_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
          model.addConstr(-x[i+1][j] + y_v1_v3 + y_v2_v3 + y_v3_v4 - y_v1_v2_v3 - y_v1_v3_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
          model.addConstr(-x[i+1][j+1] + y_v1_v4 + y_v2_v4 + y_v3_v4 - y_v1_v2_v4 - y_v1_v3_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
          model.addConstr(x[i][j] + x[i][j+1] + x[i+1][j] + x[i+1][j+1] - y_v1_v2 - y_v1_v3 - y_v1_v4 - y_v2_v3 - y_v2_v4 - y_v3_v4 + y_v1_v2_v3 + y_v1_v2_v4 + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 1)

    return model,x,y

In [11]:
def define_LP(matrix, mapping, LPversion):
    M, N = matrix.shape
    model = gp.Model(env=env)

    x = [[model.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = 1.0, name=f"x_{i}_{j}") for j in range(N)] for i in range(M)]
    y = [model.addVar(vtype=GRB.CONTINUOUS, lb = 0.0, ub = 1.0, name=f"y_{i}") for i in range(len(mapping))]

    for i in range(M - 1):
      for j in range(N - 1):
          v1, v2, v3, v4 = (i, j), (i, j + 1), (i + 1, j), (i + 1, j + 1)

          y_v1_v2_v3 = y[e(mapping, v1, v2, v3)]
          y_v1_v2_v4 = y[e(mapping, v1, v2, v4)]
          y_v1_v3_v4 = y[e(mapping, v1, v3, v4)]
          y_v2_v3_v4 = y[e(mapping, v2, v3, v4)]
          y_v1_v2_v3_v4 = y[e(mapping, v1, v2, v3, v4)]
          y_v1_v2 = y[e(mapping, v1, v2)]
          y_v1_v3 = y[e(mapping, v1, v3)]
          y_v1_v4 = y[e(mapping, v1, v4)]
          y_v2_v3 = y[e(mapping, v2, v3)]
          y_v2_v4 = y[e(mapping, v2, v4)]
          y_v3_v4 = y[e(mapping, v3, v4)]
          if LPversion == "SL" or LPversion == "flower" or LPversion == "run":
            model.addConstr(y_v1_v2 <= x[i][j])
            model.addConstr(y_v1_v2 <= x[i][j + 1])
            model.addConstr(y_v1_v2 >= x[i][j] + x[i][j + 1] - 1)
            model.addConstr(y_v1_v3 <= x[i][j])
            model.addConstr(y_v1_v3 <= x[i + 1][j])
            model.addConstr(y_v1_v3 >= x[i][j] + x[i + 1][j] - 1)
            model.addConstr(y_v1_v4 <= x[i][j])
            model.addConstr(y_v1_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v1_v4 >= x[i][j] + x[i + 1][j + 1] - 1)
            model.addConstr(y_v2_v3 <= x[i][j + 1])
            model.addConstr(y_v2_v3 <= x[i + 1][j])
            model.addConstr(y_v2_v3 >= x[i][j + 1] + x[i + 1][j] - 1)
            model.addConstr(y_v2_v4 <= x[i][j + 1])
            model.addConstr(y_v2_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v2_v4 >= x[i][j + 1] + x[i + 1][j + 1] - 1)
            model.addConstr(y_v3_v4 <= x[i + 1][j])
            model.addConstr(y_v3_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v3_v4 >= x[i + 1][j] + x[i + 1][j + 1] - 1)
            model.addConstr(y_v1_v2_v3 <= x[i][j])
            model.addConstr(y_v1_v2_v3 <= x[i][j + 1])
            model.addConstr(y_v1_v2_v3 <= x[i + 1][j])
            model.addConstr(y_v1_v2_v3 >= x[i][j] + x[i][j + 1] + x[i + 1][j] - 2)
            model.addConstr(y_v1_v2_v4 <= x[i][j])
            model.addConstr(y_v1_v2_v4 <= x[i][j + 1])
            model.addConstr(y_v1_v2_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v1_v2_v4 >= x[i][j] + x[i][j + 1] + x[i + 1][j + 1] - 2)
            model.addConstr(y_v1_v3_v4 <= x[i][j])
            model.addConstr(y_v1_v3_v4 <= x[i + 1][j])
            model.addConstr(y_v1_v3_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v1_v3_v4 >= x[i][j] + x[i + 1][j] + x[i + 1][j + 1] - 2)
            model.addConstr(y_v2_v3_v4 <= x[i][j + 1])
            model.addConstr(y_v2_v3_v4 <= x[i + 1][j])
            model.addConstr(y_v2_v3_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v2_v3_v4 >= x[i][j + 1] + x[i + 1][j] + x[i + 1][j + 1] - 2)
            model.addConstr(y_v1_v2_v3_v4 <= x[i][j])
            model.addConstr(y_v1_v2_v3_v4 <= x[i][j + 1])
            model.addConstr(y_v1_v2_v3_v4 <= x[i + 1][j])
            model.addConstr(y_v1_v2_v3_v4 <= x[i + 1][j + 1])
            model.addConstr(y_v1_v2_v3_v4 >= x[i][j] + x[i][j + 1] + x[i + 1][j] + x[i + 1][j + 1] - 3)
          if LPversion == "flower" or LPversion == "run":
            model.addConstr(y_v1_v2 <= y_v1_v2_v3 - x[i+1][j] + 1)
            model.addConstr(y_v1_v2_v3 <= y_v1_v2)
            model.addConstr(y_v1_v2 <= y_v1_v2_v4 - x[i+1][j+1] + 1)
            model.addConstr(y_v1_v2_v4 <= y_v1_v2)
            model.addConstr(y_v1_v3 <= y_v1_v2_v3 - x[i][j+1] + 1)
            model.addConstr(y_v1_v2_v3 <= y_v1_v3)
            model.addConstr(y_v1_v3 <= y_v1_v3_v4 - x[i+1][j+1] + 1)
            model.addConstr(y_v1_v3_v4 <= y_v1_v3)
            model.addConstr(y_v1_v4 <= y_v1_v2_v4 - x[i][j+1] + 1)
            model.addConstr(y_v1_v2_v4 <= y_v1_v4)
            model.addConstr(y_v1_v4 <= y_v1_v3_v4 - x[i+1][j] + 1)
            model.addConstr(y_v1_v3_v4 <= y_v1_v4)
            model.addConstr(y_v2_v3 <= y_v1_v2_v3 - x[i][j] + 1)
            model.addConstr(y_v1_v2_v3 <= y_v2_v3);
            model.addConstr(y_v2_v3 <= y_v2_v3_v4 - x[i+1][j+1] + 1)
            model.addConstr(y_v2_v3_v4 <= y_v2_v3)
            model.addConstr(y_v2_v4 <= y_v1_v2_v4 - x[i][j] + 1)
            model.addConstr(y_v1_v2_v4 <= y_v2_v4)
            model.addConstr(y_v2_v4 <= y_v2_v3_v4 - x[i+1][j] + 1)
            model.addConstr(y_v2_v3_v4 <= y_v2_v4)
            model.addConstr(y_v3_v4 <= y_v1_v3_v4 - x[i][j] + 1)
            model.addConstr(y_v1_v3_v4 <= y_v3_v4)
            model.addConstr(y_v3_v4 <= y_v2_v3_v4 - x[i][j+1] + 1)
            model.addConstr(y_v2_v3_v4 <= y_v3_v4)

            model.addConstr(y_v1_v2_v3 <= y_v1_v2_v3_v4 - x[i+1][j+1] + 1)
            model.addConstr(y_v1_v2_v3_v4 <= y_v1_v2_v3)
            model.addConstr(y_v1_v2_v4 <= y_v1_v2_v3_v4 - x[i+1][j] + 1)
            model.addConstr(y_v1_v2_v3_v4 <= y_v1_v2_v4)
            model.addConstr(y_v1_v3_v4 <= y_v1_v2_v3_v4 - x[i][j+1] + 1)
            model.addConstr(y_v1_v2_v3_v4 <= y_v1_v3_v4)
            model.addConstr(y_v2_v3_v4 <= y_v1_v2_v3_v4 - x[i][j] + 1)
            model.addConstr(y_v1_v2_v3_v4 <= y_v2_v3_v4)

            model.addConstr(y_v1_v2 + y_v3_v4 - y_v1_v2_v3_v4 <= 1)
            model.addConstr(y_v1_v3 + y_v2_v4 - y_v1_v2_v3_v4 <= 1)
            model.addConstr(y_v1_v4 + y_v2_v3 - y_v1_v2_v3_v4 <= 1)
          if LPversion == "run":
            model.addConstr(-x[i][j] + y_v1_v2 + y_v1_v3 - y_v1_v2_v3 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2 + y_v2_v3 - y_v1_v2_v3 <= 0)
            model.addConstr(-x[i+1][j] + y_v1_v3 + y_v2_v3 - y_v1_v2_v3 <= 0)
            model.addConstr(-x[i][j] + y_v1_v2 + y_v1_v4 - y_v1_v2_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2 + y_v2_v4 - y_v1_v2_v4 <= 0)
            model.addConstr(-x[i+1][j+1] + y_v1_v4 + y_v2_v4 - y_v1_v2_v4 <= 0)
            model.addConstr(-x[i][j] + y_v1_v3 + y_v1_v4 - y_v1_v3_v4 <= 0)
            model.addConstr(-x[i+1][j] + y_v1_v3 + y_v3_v4 - y_v1_v3_v4 <= 0)
            model.addConstr(-x[i+1][j+1] + y_v1_v4 + y_v3_v4 - y_v1_v3_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v2_v3 + y_v2_v4 - y_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j] + y_v2_v3 + y_v3_v4 - y_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j+1] + y_v2_v4 + y_v3_v4 - y_v2_v3_v4 <= 0)

            model.addConstr(-x[i][j] + y_v1_v2_v3 + y_v1_v2_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2_v3 + y_v1_v2_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j] + y_v1_v2_v3 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j] + y_v1_v2_v3 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2_v3 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i + 1][j] + y_v1_v2_v3 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j] + y_v1_v2_v4 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i + 1][j+1] + y_v1_v2_v4 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i + 1][j+1] + y_v1_v2_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j] + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i + 1][j+1] + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
          if LPversion == "clique":
            model.addConstr(y_v1_v2_v3 - y_v1_v2_v3_v4 >= 0)
            model.addConstr(y_v1_v2_v4 - y_v1_v2_v3_v4 >= 0)
            model.addConstr(y_v1_v3_v4 - y_v1_v2_v3_v4 >= 0)
            model.addConstr(y_v2_v3_v4 - y_v1_v2_v3_v4 >= 0)
            model.addConstr(-y_v1_v2 + y_v1_v2_v3 + y_v1_v2_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-y_v1_v3 + y_v1_v2_v3 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-y_v1_v4 + y_v1_v2_v4 + y_v1_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-y_v2_v3 + y_v1_v2_v3 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-y_v2_v4 + y_v1_v2_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-y_v3_v4 + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j] + y_v1_v2 + y_v1_v3 + y_v1_v4 - y_v1_v2_v3 - y_v1_v2_v4 - y_v1_v3_v4 + y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i][j+1] + y_v1_v2 + y_v2_v3 + y_v2_v4 - y_v1_v2_v3 - y_v1_v2_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j] + y_v1_v3 + y_v2_v3 + y_v3_v4 - y_v1_v2_v3 - y_v1_v3_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
            model.addConstr(-x[i+1][j+1] + y_v1_v4 + y_v2_v4 + y_v3_v4 - y_v1_v2_v4 - y_v1_v3_v4 - y_v2_v3_v4 + y_v1_v2_v3_v4 <= 0)
            model.addConstr(x[i][j] + x[i][j+1] + x[i+1][j] + x[i+1][j+1] - y_v1_v2 - y_v1_v3 - y_v1_v4 - y_v2_v3 - y_v2_v4 - y_v3_v4 + y_v1_v2_v3 + y_v1_v2_v4 + y_v1_v3_v4 + y_v2_v3_v4 - y_v1_v2_v3_v4 <= 1)

    return model,x,y

In [12]:
def optimizer(model, x, y, obj_coe, weight, perturbated_matrix, matrix, mapping):

    M, N = matrix.shape
    obj = gp.LinExpr()
    for i in range(M):
        for j in range(N):
            if perturbated_matrix[i][j] == 0:
                obj += (-weight) * x[i][j]
            else:
                obj += weight * x[i][j]

    for i in range(M - 1):
        for j in range(N - 1):
            v1 = (i, j)
            v2 = (i, j + 1)
            v3 = (i + 1, j)
            v4 = (i + 1, j + 1)

            obj += obj_coe[0] * x[i][j]
            obj += obj_coe[1] * x[i][j + 1]
            obj += obj_coe[2] * x[i + 1][j]
            obj += obj_coe[3] * x[i + 1][j + 1]

            obj += obj_coe[4] * y[e(mapping, v1, v4)]
            obj += obj_coe[5] * y[e(mapping, v2, v3)]
            obj += obj_coe[6] * y[e(mapping, v1, v2)]
            obj += obj_coe[7] * y[e(mapping, v2, v4)]
            obj += obj_coe[8] * y[e(mapping, v3, v4)]
            obj += obj_coe[9] * y[e(mapping, v1, v3)]
            obj += obj_coe[10] * y[e(mapping, v1, v2, v3)]
            obj += obj_coe[11] * y[e(mapping, v1, v2, v4)]
            obj += obj_coe[12] * y[e(mapping, v1, v3, v4)]
            obj += obj_coe[13] * y[e(mapping, v2, v3, v4)]
            obj += obj_coe[14] * y[e(mapping, v1, v2, v3, v4)]

    model.setObjective(obj, GRB.MAXIMIZE)
    # model.setParam('OutputFlag', 0)
    # Optimize the model

    start = time.perf_counter()
    model.optimize()
    elapsed = time.perf_counter() - start

    tolerance = 1e-6  # Define your tolerance value
    is_binary = all(
        abs(x[i][j].X - 0) <= tolerance or abs(x[i][j].X - 1) <= tolerance
        for i in range(M) for j in range(N)
    )

    return {
        "is_binary": is_binary,
        "total_wall_time": elapsed,
        "objective_value": model.objVal if model.status == GRB.OPTIMAL else None,
    }

## Run Experiments

In [16]:
def run_experiment(input_file, num_trials):
    pattern_potential = [10, 20, 30, 40]
    obj_coe = compute_potential(pattern_potential)
    weight = 25
    clean_matrix, mapping = read_data(input_file)
    M, N = clean_matrix.shape

    models = {
        'clique_IP': define_clique_IP_model(clean_matrix, mapping),
        'clique_LP': define_LP(clean_matrix, mapping, "clique"),
        'std_LP': define_LP(clean_matrix, mapping, "SL"),
        'flower_LP': define_LP(clean_matrix, mapping, "flower"),
        'run_LP': define_LP(clean_matrix, mapping, "run")
    }

    num_models = len(models)
    results = {
        'tightness': np.zeros((num_models, 5, num_trials)),
        'time': np.zeros((num_models, 5, num_trials)),
        'obj': np.zeros((num_models, 5, num_trials))
    }

    for p_idx, p in enumerate([0.1, 0.2, 0.3, 0.4, 0.5]):
        for trial in range(num_trials):
            # Generate noisy matrix
            noisy_matrix = clean_matrix + (np.random.rand(M, N) < p) * (1 - 2 * clean_matrix)

            # Run optimization for each model
            for model_idx, (model_name, (model, x, y)) in enumerate(models.items()):
                result = optimizer(model, x, y, obj_coe, weight, noisy_matrix, clean_matrix, mapping)

                results['tightness'][model_idx, p_idx, trial] = result["is_binary"]
                results['time'][model_idx, p_idx, trial] = result["total_wall_time"]
                results['obj'][model_idx, p_idx, trial] = result["objective_value"]

    return tuple(results.values())

In [ ]:
run_experiment('/content/CEN_15.txt', 50)